In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/reg_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-08-20 19:23:22.746146


In [25]:
# Initial full list
regs = {}
pathdir = "DataStore/2018-08/Reg0820/"
reg_f = "reg_model_0820.pkl"
# Similarity regressions

for dm in ["ldavecs", "docvecs"]:
    sim_regs = [
        # Main results
        
        # No FE
        #0
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear)".format(dm),
        
        # Field FE
        #1
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_naics_name)".format(dm),
        #2
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass)".format(dm),
        
        # Inventor FE
        #3
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_naics_name) + C(common_est_inv) + C(common_pat_inv)"\
        .format(dm),
        #4
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv)"\
        .format(dm),
        
        # Lawyer FE
        #5
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_naics_name) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match)".format(dm),
        #6
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match)".format(dm),
        
        # All FE: No Tech Sim
        #7
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_naics_name) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match)".format(dm),
        #8
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match)".format(dm),
        
        # Sim: No FE
        #9
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + sim_med_{0}_pc".format(dm),
        #10
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + sim_med_{0}_pc_msa".format(dm),
        #11
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc + C(tp_gyear) ".format(dm),
        #12
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc_msa + C(tp_gyear)".format(dm),
        
        # Sim: PC FE
        #13
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + sim_med_{0}_pc".format(dm),
        #14
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + sim_med_{0}_pc_msa".format(dm),
        #15
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc + C(tp_gyear) + C(tp_primclass) ".format(dm),
        #16
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc_msa + C(tp_gyear) + C(tp_primclass)".format(dm),
        
        # Sim: Inventor FE
        #17
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        sim_med_{0}_pc".format(dm),
        #18
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        sim_med_{0}_pc_msa".format(dm),
        #19
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) +\
        C(common_pat_inv) ".format(dm),
        #20
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc_msa + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) +\
        C(common_pat_inv)".format(dm),
        
        # Sim: Lawyer FE
        #21
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + sim_med_{0}_pc".format(dm),
        #22
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + sim_med_{0}_pc_msa".format(dm),
        #23
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) ".format(dm),
        #24
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc_msa + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) ".format(dm),
        
        # Sim: All Fixed Effects
        #25
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match) + sim_med_{0}_pc".format(dm),
        #26
        "sim_{0} ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match) + sim_med_{0}_pc_msa".format(dm),
        #27
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match) ".format(dm),
        #28
        "sim_{0} ~ C(inv_msa_match)*sim_med_{0}_pc_msa + C(tp_gyear) + C(tp_primclass) + C(common_est_inv) + C(common_pat_inv) + \
        C(lawyer_match) + C(primclass_match)".format(dm),
    ]
    
    # Add mean as well
    mean_sim_regs = [i.replace("med_", "mean_") for i in sim_regs]
    # Number of median equations
    n_med_eqns = len(sim_regs)
    sim_regs = sim_regs+mean_sim_regs
    # Replace sim with norm
    n_sim_regs = [i.replace("sim_","norm_sim_") for i in sim_regs]
    # Number of all equations in sim_regs
    n_eqns = len(sim_regs)
    # Update regression list
    sim_regs = sim_regs+n_sim_regs
    
    # NAICS indices
    regs[dm, "all_models"] = pd.Series(sim_regs)
    regs[dm, "naics_name"] = regs[dm, "all_models"]
    
    # PC indices
    pc_ind = [0,2,4,6,10,12,14,16,18,20,22,24]
    # Add mean vequations
    pc_ind.extend([i+n_med_eqns for i in pc_ind])
    # Add norm equations
    pc_ind.extend([i+n_eqns for i in pc_ind])
    regs[dm, "primclass"] = regs[dm, "all_models"].loc[pc_ind]
    mn = [
        "Year FE", #0
        "NAICS FE", #1
        "PC FE", #2
        "Inv FE NAICS", #3
        "Inv FE PC", #4    
        "Law FE NAICS", #5
        "Law FE PC", #6  
        "All FE NAICS", #7
        "All FE PC", #8 
        "Sim PC", #9
        "Sim MSA PC", #10
        "Int PC", #11
        "Int MSA PC", #12
        "PC FE - Sim PC", #13
        "PC FE - Sim MSA PC", #14
        "PC FE - Int PC", #15
        "PC FE - Int MSA PC", #16
        "Inv FE - Sim PC", #17
        "Inv FE - Sim MSA PC", #18
        "Inv FE - Int PC", #19
        "Inv FE - Int MSA PC", #20
        "Law FE - Sim PC", #21
        "Law FE - Sim MSA PC", #22
        "Law FE - Int PC", #23
        "Law FE - Int MSA PC", #24
        "All FE - Sim PC", #25
        "All FE - Sim MSA PC", #26
        "All FE - Int PC", #27
        "All FE - Int MSA PC", #28
    ]
    # Add median
    mn = [i+" - Md" for i in mn]
    # Add mean
    mn1 = [i.replace("Md", "Mn") for i in mn]
    # Add both together
    mn = mn+mn1
    # Add norm
    mn2 = ["N "+i for i in mn]
    # Extend
    mn.extend(mn2)
        
    regs["model_names"] = pd.Series(mn)
pickle.dump(regs, open(pathdir+reg_f, "wb"))


In [27]:
info_dict = {'$N$':lambda x: "{0:d}".format(int(x.nobs)),
'Adjusted $R^2$':lambda x: "{:.2f}".format(x.rsquared_adj)}
def get_fit(formula, grouped_data, group_col, cov_type, return_fit = False):
    summ = []
    tables = {}
    
    # Remove missing values used in formula
    col_used = re.findall('\((.*?)\)',formula)
    grouped_data = grouped_data.dropna(how="any", subset=col_used) # Should take care of gyear > 1980 issues
    
    # Group and then get results
    grouped_data = grouped_data.groupby(group_col)
    
    for n,g in grouped_data:
        try:
            fit = smf.ols(formula = formula, data = g, missing="drop").fit(cov_type=cov_type)
            # Get results tables
            tables[n] = fit.summary2().tables
            # Append results
            summ.append(fit)
        except Exception:
            print(n)
            print("Regression error")
            pass
    # Get full results output
    # Dataframe of full results
    res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    
    # Get partial results
    # 1. Get relevant variables from index of full results: UPDATED
    regressors = [v for v in res_no_stars.index.unique() if ("sim_" in v) | ("match" in v) | ("common_" in v)]
    # 2. Make sure regressors come last
    regressors = regressors+["Intercept"]
    # 3. Get results with regressors
    part_res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    part_res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    
    # 4. Get index of where Intercept is and add 2 (to include standard error)
    last_ind = list(part_res_stars.index).index("Intercept")+2
    
    # 5. Get partial results
    part_res_no_stars = pd.concat([part_res_no_stars.iloc[:last_ind], part_res_no_stars.iloc[-2::]])
    part_res_stars = pd.concat([part_res_stars.iloc[:last_ind], part_res_stars.iloc[-2::]])
    
    if return_fit == True:
        return summ, tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    else:
        return tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars

In [ ]:
# Regressions for random sample
dms = ["ldavecs", "docvecs"]
mod_keys = ["naics_name", "primclass"]
# mod_keys = ["naics_name"]
files = ["reg_naics_name_sim_tr_0726.parq", "reg_primclass_sim_tr_0726.parq"]
f_dict = dict(zip(mod_keys,files))
file_pathdir = "DataStore/2018-07-P2/Reg0726/"

for k in mod_keys:
    f = f_dict[k]
    
    # Merge common inventor data
    rs = fastparquet.ParquetFile(file_pathdir+f).to_pandas()
    rs2 = fastparquet.ParquetFile(file_pathdir+"{0}_common_inv_0726.parq".format(k))\
    .to_pandas(["tp", "op", "common_est_inv", "common_pat_inv", "lawyer_match"])
    rs = rs.merge(rs2, how="left", on = ["tp", "op"])
    del(rs2)
    print(rs.columns)
    
    for dm in dms:
        
        print((dm, k, "started"))
        print(datetime.datetime.now())
        # 3. Define output
        samp_out = {}
        formulas = list(regs[dm,k])
        formulas_ind = list(regs[dm,k].index)
        cov = "HC1"
        for i, j in zip(formulas_ind, formulas):
            print(k)
            print(i)
            print(j)
            print(datetime.datetime.now())
            try:
                out = get_fit(j, rs, "year_group", cov, return_fit = False)
                samp_out[i] = {}
                samp_out[i]["model"] = j
                samp_out[i]["tables"] = out[0]
                samp_out[i]["res_no_stars"] = out[1]
                samp_out[i]["res_stars"] = out[2]
                samp_out[i]["part_res_no_stars"] = out[3]
                samp_out[i]["part_res_stars"] = out[4]
            except Exception as e:
                logging.exception("error here")
                pass
            print("finished")
            print(datetime.datetime.now())
        # Define outfile
        o_f = "reg_{0}_{1}_out_0820.pkl".format(k,dm)
        pickle.dump(samp_out, open(pathdir+o_f, "wb"))    

INFO:root:Index(['tp', 'op', 'sim_ldavecs', 'sim_docvecs', 'tp_gyear', 'tp_naics_name',
       'tp_primclass', 'tp_inv_msa', 'op_naics_name', 'op_primclass',
       'op_inv_msa', 'inv_msa_match', 'primclass_match', 'norm_sim_ldavecs',
       'norm_sim_docvecs', 'sim_med_ldavecs_naics_msa',
       'sim_mean_ldavecs_naics_msa', 'sim_med_docvecs_naics_msa',
       'sim_mean_docvecs_naics_msa', 'sim_med_ldavecs_pc',
       'sim_mean_ldavecs_pc', 'sim_med_docvecs_pc', 'sim_mean_docvecs_pc',
       'sim_med_ldavecs_pc_msa', 'sim_mean_ldavecs_pc_msa',
       'sim_med_docvecs_pc_msa', 'sim_mean_docvecs_pc_msa', 'year_group',
       'norm_norm_sim_ldavecs', 'norm_norm_sim_docvecs',
       'norm_sim_med_ldavecs_naics_msa', 'norm_sim_mean_ldavecs_naics_msa',
       'norm_sim_med_docvecs_naics_msa', 'norm_sim_mean_docvecs_naics_msa',
       'norm_sim_med_ldavecs_pc', 'norm_sim_mean_ldavecs_pc',
       'norm_sim_med_docvecs_pc', 'norm_sim_mean_docvecs_pc',
       'norm_sim_med_ldavecs_pc_msa', 'nor

### Results

In [29]:
dms = ["ldavecs", "docvecs"]
mod_keys = ["naics_name", "primclass"]
pathdir = "DataStore/2018-08/Reg0820/"
reg_f = "reg_model_0820.pkl"
regs = pickle.load(open(pathdir+reg_f, "rb"))
tab = {}
for dm in dms:
    for g in mod_keys:
        
        f = f_dict[g]
        o_f = "reg_{0}_{1}_out_0820.pkl".format(g,dm)
        
        res = pickle.load(open(pathdir+o_f, "rb"))

        res_out = pd.DataFrame()

        for k in res.keys():
            lks = res[k]["model"].split(" ~ ")[0]
            
            # Selecting portion of results without intercept
            cdf = res[k]["part_res_stars"].reset_index()
            ic_ind = cdf.loc[cdf["index"] == "Intercept"].index[0]
            # Include N & R^2
            cdf = cdf.iloc[pd.np.r_[0:ic_ind,ic_ind+2:len(cdf)]]
            cdf["Model"] = regs["model_names"][k]
            cdf["Model Num"] = k
            cdf["LKS"] = lks
        
            res_out = res_out.append(cdf)
        
#         res_out = res_out.replace("C(inv_msa_match)[T.True]", "$I(MSA \, Match)$")
#         res_out = res_out.replace("C(tp_from_uni)[T.True]", "$I(tp \, from \, uni)$")
#         res_out["Sample"] = samp_name[f]
        
#         m_ind = res_out["Model"].drop_duplicates().index.tolist()
#         res_out.loc[~(res_out.index.isin(m_ind)), ["LKS", "Sample"]] = ""
        
        tab[dm,g] = res_out
        del(res_out)
        
full_tab = tab

In [30]:
r = pd.DataFrame()
for k,v in tab.items():
    print(k)
    display(v)
    v["samp"] = str(k)
    v = v.reset_index(drop=True)
    r = pd.concat([r,v],axis=0)

INFO:root:('ldavecs', 'naics_name')


,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index
0,0.0069***,0.0119***,0.0219***,0.0204***,sim_ldavecs,Year FE - Md,0,C(inv_msa_match)[T.True]
1,(0.0012),(0.0009),(0.0008),(0.0007),sim_ldavecs,Year FE - Md,0,
4,193173,281625,438317,570099,sim_ldavecs,Year FE - Md,0,$N$
5,0.00,0.00,0.00,0.00,sim_ldavecs,Year FE - Md,0,Adjusted $R^2$
0,0.0102***,0.0137***,0.0224***,0.0185***,sim_ldavecs,NAICS FE - Md,1,C(inv_msa_match)[T.True]
1,(0.0011),(0.0009),(0.0008),(0.0007),sim_ldavecs,NAICS FE - Md,1,
4,193173,281625,438317,570099,sim_ldavecs,NAICS FE - Md,1,$N$
5,0.08,0.08,0.10,0.11,sim_ldavecs,NAICS FE - Md,1,Adjusted $R^2$
0,0.0103***,0.0138***,0.0217***,0.0178***,sim_ldavecs,PC FE - Md,2,C(inv_msa_match)[T.True]
1,(0.0011),(0.0009),(0.0007),(0.0007),sim_ldavecs,PC FE - Md,2,


INFO:root:('ldavecs', 'primclass')


,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index
0,0.0104***,0.0126***,0.0177***,0.0154***,sim_ldavecs,Year FE - Md,0,C(inv_msa_match)[T.True]
1,(0.0015),(0.0012),(0.0010),(0.0008),sim_ldavecs,Year FE - Md,0,
4,171099,252478,402006,530245,sim_ldavecs,Year FE - Md,0,$N$
5,0.00,0.00,0.00,0.00,sim_ldavecs,Year FE - Md,0,Adjusted $R^2$
0,0.0139***,0.0158***,0.0153***,0.0106***,sim_ldavecs,PC FE - Md,2,C(inv_msa_match)[T.True]
1,(0.0014),(0.0011),(0.0009),(0.0007),sim_ldavecs,PC FE - Md,2,
4,171099,252478,402006,530245,sim_ldavecs,PC FE - Md,2,$N$
5,0.18,0.18,0.20,0.20,sim_ldavecs,PC FE - Md,2,Adjusted $R^2$
0,0.0007,0.0127***,0.0128***,0.0121***,sim_ldavecs,Inv FE PC - Md,4,C(common_est_inv)[T.1.0]
1,(0.0033),(0.0018),(0.0012),(0.0009),sim_ldavecs,Inv FE PC - Md,4,


INFO:root:('docvecs', 'naics_name')


,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index
0,0.0025***,0.0070***,0.0078***,0.0076***,sim_docvecs,Year FE - Md,0,C(inv_msa_match)[T.True]
1,(0.0007),(0.0006),(0.0005),(0.0004),sim_docvecs,Year FE - Md,0,
4,192841,281222,437685,569252,sim_docvecs,Year FE - Md,0,$N$
5,0.00,0.00,0.00,0.00,sim_docvecs,Year FE - Md,0,Adjusted $R^2$
0,0.0042***,0.0080***,0.0083***,0.0079***,sim_docvecs,NAICS FE - Md,1,C(inv_msa_match)[T.True]
1,(0.0007),(0.0006),(0.0005),(0.0004),sim_docvecs,NAICS FE - Md,1,
4,192841,281222,437685,569252,sim_docvecs,NAICS FE - Md,1,$N$
5,0.06,0.06,0.07,0.04,sim_docvecs,NAICS FE - Md,1,Adjusted $R^2$
0,0.0044***,0.0082***,0.0082***,0.0077***,sim_docvecs,PC FE - Md,2,C(inv_msa_match)[T.True]
1,(0.0007),(0.0006),(0.0005),(0.0004),sim_docvecs,PC FE - Md,2,


INFO:root:('docvecs', 'primclass')


,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index
0,0.0067***,0.0108***,0.0116***,0.0083***,sim_docvecs,Year FE - Md,0,C(inv_msa_match)[T.True]
1,(0.0009),(0.0007),(0.0005),(0.0005),sim_docvecs,Year FE - Md,0,
4,170882,252174,401623,529686,sim_docvecs,Year FE - Md,0,$N$
5,0.00,0.00,0.00,0.00,sim_docvecs,Year FE - Md,0,Adjusted $R^2$
0,0.0073***,0.0111***,0.0109***,0.0079***,sim_docvecs,PC FE - Md,2,C(inv_msa_match)[T.True]
1,(0.0009),(0.0007),(0.0005),(0.0004),sim_docvecs,PC FE - Md,2,
4,170882,252174,401623,529686,sim_docvecs,PC FE - Md,2,$N$
5,0.07,0.07,0.08,0.07,sim_docvecs,PC FE - Md,2,Adjusted $R^2$
0,0.0022,0.0093***,0.0074***,0.0084***,sim_docvecs,Inv FE PC - Md,4,C(common_est_inv)[T.1.0]
1,(0.0022),(0.0012),(0.0007),(0.0005),sim_docvecs,Inv FE PC - Md,4,


In [31]:
r.to_csv(pathdir+"reg_pairs_out_0820.csv")